# Connecting to Drive

In [ ]:
# Connecting to google drive
from google.colab import drive

drive.mount("/content/drive")

In [ ]:
# Path
%cd "/content/drive/MyDrive/Bridge construction health/brridge-failure-detection"

In [ ]:
! ls -ltrh

# Imports and installations

In [ ]:
import os
import shutil
from pathlib import Path

In [ ]:
from PIL import Image
import numpy as np

# Unzipping the dataset of CWT images

In [ ]:
! unrar x data.rar ./

In [ ]:
! unrar x data1.rar ./data1

# Merging datasets

In [ ]:
def merge_directories(dir1, dir2, output_dir):
    # Convert paths to Path objects
    dir1 = Path(dir1)
    dir2 = Path(dir2)
    output_dir = Path(output_dir)

    # Create output directory if it doesn't exist
    output_dir.mkdir(parents=True, exist_ok=True)

    # Get a list of all subdirectories in both input directories
    subdirs = set(os.listdir(dir1)).union(set(os.listdir(dir2)))

    # Iterate over each subdirectory
    for subdir in subdirs:
        src_subdir1 = dir1 / subdir
        src_subdir2 = dir2 / subdir
        dst_subdir = output_dir / subdir

        # If the subdir exists in the first source directory, copy it to the output
        if src_subdir1.exists():
            shutil.copytree(src_subdir1, dst_subdir, dirs_exist_ok=True)

        # If the subdir exists in the second source directory, merge it with the output
        if src_subdir2.exists():
            for item in src_subdir2.iterdir():
                src_item = src_subdir2 / item.name
                dst_item = dst_subdir / item.name

                if src_item.is_dir():
                    # If it's a directory, copy it recursively
                    shutil.copytree(src_item, dst_item, dirs_exist_ok=True)
                else:
                    # If it's a file, copy it, overwriting if necessary
                    shutil.copy2(src_item, dst_item)

In [ ]:
dir1 = "./data"
dir2 = "./data1/data1"
output_dir = "./main_datatset"

merge_directories(dir1, dir2, output_dir)

# Vectorization of images

In [ ]:
def vectorize_rgb_image_to_array(png_path):
    # Load the PNG image
    image = Image.open(png_path)

    # Convert the image to RGB mode (if it's not already)
    image = image.convert('RGB')

    # Convert the image to a numpy array
    img_array = np.array(image)

    return img_array

In [ ]:
! ls "./main_datatset/After_reconstruction" | head

1_After_reconstruction_0_1.png
1_After_reconstruction_0_2.png
1_After_reconstruction_0_3.png
1_After_reconstruction_0_4.png
1_After_reconstruction_0_5.png
1_After_reconstruction_100_1.png
1_After_reconstruction_100_2.png
1_After_reconstruction_100_3.png
1_After_reconstruction_100_4.png
1_After_reconstruction_100_5.png


## Checking the sample number of each class

In [ ]:
len(os.listdir("./main_datatset/When_rebuilding"))

1037

In [ ]:
len(os.listdir("./main_datatset/Before_renovation"))

1033

In [ ]:
len(os.listdir("./main_datatset/After_reconstruction"))

1024

## Vetorizing all

In [ ]:
X = []
y = []

In [ ]:
all_classes = os.listdir("./main_datatset/")

In [ ]:
all_classes_dict = {
    "Before_renovation": 0,
    "After_reconstruction": 1,
    "When_rebuilding": 2
}

In [ ]:
# Vectorizing and added labels of images
for lbl_class in all_classes:
  print(lbl_class)
  images = os.listdir(f"./main_datatset/{lbl_class}")

  for img in images:
    img_path = f"./main_datatset/{lbl_class}/{img}"
    img_array = vectorize_rgb_image_to_array(img_path)
    X.append(img_array)
    y.append(all_classes_dict[lbl_class])

When_rebuilding
Before_renovation
After_reconstruction


# Model Architecture

In [ ]:
from sklearn.model_selection import train_test_split

X = np.array(X)
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
X_train.shape

(2475, 23, 23, 3)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(23, 23, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' for one-hot encoded labels
              metrics=['accuracy'])

# Print model summary
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 21, 21, 32)          │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 10, 10, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 8, 8, 64)            │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 4, 4, 64)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 1024)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │          65,600 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 3)                   │             195 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 85,187 (332.76 KB)

 Trainable params: 85,187 (332.76 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Define the model
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(23, 23, 3)),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' for one-hot encoded labels
              metrics=['accuracy'])

# Print model summary
model.summary()

In [ ]:
# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - accuracy: 0.3397 - loss: 8.0219 - val_accuracy: 0.3893 - val_loss: 1.2618
Epoch 2/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.3960 - loss: 1.1894 - val_accuracy: 0.4653 - val_loss: 1.0341
Epoch 3/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.4517 - loss: 1.0674 - val_accuracy: 0.4620 - val_loss: 1.0341
Epoch 4/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 31ms/step - accuracy: 0.4990 - loss: 0.9958 - val_accuracy: 0.5412 - val_loss: 0.9475
Epoch 5/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.5714 - loss: 0.9184 - val_accuracy: 0.5412 - val_loss: 0.9498
Epoch 6/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.5931 - loss: 0.8551 - val_accuracy: 0.6155 - val_loss: 0.8163
Epoch 7/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 22ms/step - accuracy: 0.6434 - loss: 0.8164 - val_accuracy: 0.6446 - val_loss: 0.7820
Epoch 8/100
78/78 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.6707 - loss: 0.7654 - val_accuracy: 0.

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f'Test loss: {test_loss}')
print(f'Test accuracy: {test_accuracy}')

20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7947 - loss: 1.7399
Test loss: 1.781162977218628
Test accuracy: 0.783521831035614
